In [3]:
%%bash
pwd
# ensure that pip install pyspark==3.2.1 i.e. the SPARK version matches the pyspark version, otherwise feast apply fails with errorr "blah blah encryption" 

/home/cdsw/scripts


In [ ]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore
import os

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
# feature_store = FeatureStore('/home/cdsw/homecredit_feature_repo')  # Initialize the feature store

# entity_df = pd.DataFrame.from_dict(
#     {
#         # entity's join key -> entity values
#         "sk_id_curr": [100002, 100003, 100004],
#         # "event_timestamp" (reserved key) -> timestamps
#         "event_timestamp": [
#             datetime(2022, 12, 6, 12, 33, 20),
#             datetime(2022, 12, 6, 12, 33, 20),
#             datetime(2022, 12, 6, 12, 33, 20),
#         ],
#     }
# )

## Alternatively


# You can also pass a SQL string to generate the above dataframe.
# This is useful for getting all entities in a timeframe from some data source.

store = FeatureStore(repo_path='/home/cdsw/homecredit_feature_repo')

# entity_df = "SELECT sk_id_curr, current_timestamp as event_timestamp FROM  homecredit_processed_data \
#             ORDER BY event_timestamp DESC"


entity_sql = f"""
    SELECT
        sk_id_curr,
        event_timestamp
    FROM {store.get_data_source("homecredit_processed_data").get_table_query_string()}
    WHERE event_timestamp < current_timestamp
"""



training_df = store.get_historical_features(
    entity_df=entity_df,

    features=[
        "homecredit_appl_details_view:target",
        # "homecredit_appl_details_view:flag_own_car",
        # "homecredit_appl_details_view:flag_own_realty",
    ]
).to_df()

feature_service = store.get_feature_service("homecredit_appl_details_Fservice")
training_df=store.get_historical_features(features=feature_service, entity_df = entity_df).to_df()
print("----- Example features -----\n")
print(training_df.head())

In [23]:
print(training_df.shape)

(356251, 717)


In [ ]:
training_df.show()

In [24]:
training_df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356251 entries, 0 to 356250
Columns: 717 entries, sk_id_curr to cc_count
dtypes: datetime64[ns](1), float64(715), int32(1)
memory usage: 1.9 GB
